In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import os
import pandas as pd
import numpy as np
import datetime

In [2]:
data = pd.read_excel(os.path.join('..', 'Olympics.xls'))
data

,Gender,Event,Location,Year,Medal,Name,Nationality,Result
0,M,10000M Men,Rio,2016,G,Mohamed FARAH,USA,1505
1,M,10000M Men,Rio,2016,S,Paul Kipngetich TANUI,KEN,1626
2,M,10000M Men,Rio,2016,B,Tamirat TOLA,ETH,1626
3,M,10000M Men,Beijing,2008,G,Kenenisa BEKELE,ETH,1621
4,M,10000M Men,Beijing,2008,S,Sileshi SIHINE,ETH,1623
...,...,...,...,...,...,...,...,...
2389,W,Triple Jump Women,Athens,2004,S,Hrysopiyi DEVETZI,GRE,15.25
2390,W,Triple Jump Women,Athens,2004,B,Tatyana LEBEDEVA,RUS,15.14
2391,W,Triple Jump Women,Atlanta,1996,G,Inessa KRAVETS,UKR,15.33
2392,W,Triple Jump Women,Atlanta,1996,S,Inna LASOVSKAYA,RUS,14.98


In [3]:
data = data[data.Result != "None"]

In [4]:
data = pd.get_dummies(data, columns=['Gender'])
data.head()

,Event,Location,Year,Medal,Name,Nationality,Result,Gender_M,Gender_W
0,10000M Men,Rio,2016,G,Mohamed FARAH,USA,1505,1,0
1,10000M Men,Rio,2016,S,Paul Kipngetich TANUI,KEN,1626,1,0
2,10000M Men,Rio,2016,B,Tamirat TOLA,ETH,1626,1,0
3,10000M Men,Beijing,2008,G,Kenenisa BEKELE,ETH,1621,1,0
4,10000M Men,Beijing,2008,S,Sileshi SIHINE,ETH,1623,1,0


In [5]:
#data = data.drop(columns=['Gender_M'])
#data.head()

In [6]:
# Assign X (data) and y (target)
x = data[['Result','Gender_W','Gender_M']]
y = data["Event"]
print(x.shape, y.shape)

(2351, 3) (2351,)


In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=5,test_size=0.5)

In [8]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(max_iter=1000)
classifier

LogisticRegression(max_iter=1000)

In [9]:
classifier.fit(x_train, y_train)

/Users/eobodoechine/opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(max_iter=1000)

In [10]:
print(f"Training Data Score: {classifier.score(x_train, y_train)}")
print(f"Testing Data Score: {classifier.score(x_test, y_test)}")

Training Data Score: 0.03489361702127659
Testing Data Score: 0.03231292517006803


In [11]:
predictions = classifier.predict(x_test)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   ['400M Men' '400M Men' '400M Men' '400M Men' '400M Men'
 '4X100M Relay Women' '400M Men' '400M Men' '400M Men' '400M Men']
First 10 Actual labels: ['Decathlon Men', 'Hammer Throw Men', 'Shot Put Men', 'Triple Jump Women', '100M Women', 'High Jump Women', '3000M Steeplechase Men', 'Heptathlon Women', 'Long Jump Men', 'Triple Jump Men']


In [12]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

,Prediction,Actual
0,400M Men,Decathlon Men
1,400M Men,Hammer Throw Men
2,400M Men,Shot Put Men
3,400M Men,Triple Jump Women
4,400M Men,100M Women
...,...,...
1171,400M Men,Long Jump Men
1172,400M Men,Decathlon Men
1173,400M Men,4X100M Relay Men
1174,400M Men,50Km Race Walk Men


In [13]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train, y_train)
clf.score(x_test, y_test)

0.9107142857142857

In [14]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(x_train, y_train)
rf.score(x_test, y_test)

0.9115646258503401

In [15]:
sorted(zip(rf.feature_importances_, x), reverse=True)

[(0.9634924388689579, 'Result'),
 (0.01876925551045579, 'Gender_W'),
 (0.017738305620586373, 'Gender_M')]

In [16]:
z=list(data[['Result','Gender_W','Gender_M']])

In [17]:
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rf.estimators_[5]
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rf.estimators_[5]
# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = z, rounded = True, precision = 1)
# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')
# Write graph to a png file
graph.write_png('tree.png')

In [30]:
# Limit depth of tree to 3 levels
rf_small = RandomForestClassifier(n_estimators=200, max_depth = 3)
rf_small.fit(x_train, y_train)
# Extract the small tree
tree_small = rf_small.estimators_[5]
# Save the tree as a png image
export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = z, rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('small_tree.dot')
graph.write_png('small_tree.png');